In [1]:
import numpy as np
# from matplotlib import pyplot as plt
from time import time

import pyesapi
import atexit
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

In [2]:
# !pip install chart_studio
import chart_studio.plotly as py
import plotly.figure_factory as ff
import pandas as pd

In [2]:
for pat in app.PatientSummaries:
    print(pat.Id, pat.LastName, pat.FirstName)

CCS OncQT Sunday_Rob CCS OncQT
2274444 Zlack James
Eclipse01 Holy Name PET Case 1C 
2277755 Black James
SRS-Lung Baker Josephin
00IGRT_50 Protosky Pekka
00IGRT21 Headly Nick
00IGRT71 Lundgren Larry
00IGRT67 Seddon Ron
00IGRT66 Jones Harold
00IGRT65 Pearlman Samuel
00IGRT64 Jones Melody
00IGRT41 Bladder Blake
00IGRT42 Prosser Pedro
00IGRT43 Lunn Luke
00IGRT01 Lundahl Esau
00IGRT44 Benson Brigitte
00IGRT61 Peter Pelllhum
00IGRT62 Pelton Paul
00IGRT_85 Peltz Reginald
00IGRT_87 Long Henry
00IGRT_90 Pellman Elvis
00IGRT_86 Lunsford Andy
00IGRT_88 Pelletier Richard
00IGRT_89 Denton Margaret
Eclipse_18 Rt Lung Tumor RA, DRR
MK11111 Mike Arinas
11133313 Female MR2 CCS Test
MR30928 Smythe Janine
5563207 Cote Pearl
MR1234 Demont Patricia
MR43729 Smith David
100665241 Pancher Harold
MR039547 Flannigan Patrice
2587413 James Susan T
1897856 Arrows Gail
MR09395 Newberry Sara
MR83943 O'Malee Candace
9876548 Janzen Joyce T
2563149 Jamieson Everett T
MR37438 Smyth Jannine
2587469 Jacobson Randolph T
43

In [3]:
app.ClosePatient()
patient = app.OpenPatientById('LUNG_063')
plan = patient.CoursesLot('Min MU 400').PlanSetupsLot(0)
structures = plan.StructureSet.StructuresLot()
print(f'plan id: {plan.Id}\nstructure set id: {plan.StructureSet.Id}')

plan id: MOBug5Field
structure set id: Struc


In [42]:
dfc = None

for beam in plan.Beams:
    beam_target = structures[beam.TargetStructure.Id]
    for idx, contour in enumerate(beam.GetStructureOutlines(beam_target,True)):
        dfc = pd.concat([dfc, pd.DataFrame({
            'Beam ID' : beam.Id,
            'Structure ID': beam_target.Id,
            'Color': beam_target.Color.ToString(),
            'Points X' : [p.X for p in contour],
            'Points Y' : [p.Y for p in contour],
            'Contour Idx' : idx,
        })])
dfc

,Beam ID,Structure ID,Color,Points X,Points Y,Contour Idx
0,Field 1,PTV,#FF00FFFF,-4.4,17.8,0
1,Field 1,PTV,#FF00FFFF,1.8,17.7,0
2,Field 1,PTV,#FF00FFFF,3.8,17.5,0
3,Field 1,PTV,#FF00FFFF,5.0,17.3,0
4,Field 1,PTV,#FF00FFFF,6.6,17.1,0
...,...,...,...,...,...,...
79,Field 5,PTV,#FF00FFFF,-10.4,16.5,0
80,Field 5,PTV,#FF00FFFF,-8.8,16.9,0
81,Field 5,PTV,#FF00FFFF,-7.6,17.1,0
82,Field 5,PTV,#FF00FFFF,-4.8,17.5,0


In [49]:
tdf=dfc[dfc['Beam ID'] == 'Field 1']

In [61]:
for gn, gdf in tdf.groupby('Contour Idx'):
#     print(gn)
    print(gdf['Structure ID'][0])

PTV


In [62]:
from PIL import ImageColor

ModuleNotFoundError: No module named 'PIL'

In [6]:
#multi layer
app.ClosePatient()
patient = app.OpenPatientById('LUNG_063')
plan = patient.CoursesLot('BraggRnD').PlanSetupsLot('SCPlan2')
# structures = plan.StructureSet.StructuresLot()
# print(f'plan id: {plan.Id}\nstructure set id: {plan.StructureSet.Id}')

# load spot data
df = None
print("Extracting data...")
for beam in plan.IonBeams:
    beamMetersetValue = beam.Meterset.Value
    totMetersetWeight = [cpp for cpp in beam.IonControlPoints][-1].MetersetWeight
    eParams = beam.GetEditableParameters()
    for layerIdx, controlPoint in enumerate(eParams.IonControlPointPairs):
        df = pd.concat([df, pd.DataFrame({
            'Field ID': beam.Id,
            'Energy Layer [MeV]': controlPoint.NominalBeamEnergy,
            'X [mm]': [s.X for s in controlPoint.FinalSpotList],
            'Y [mm]': [s.Y for s in controlPoint.FinalSpotList],
            'MU': [s.Weight * (beamMetersetValue / totMetersetWeight) for s in controlPoint.FinalSpotList],
        })])

Extracting data...


In [22]:
for fid, fgrp in df.groupby('Field ID'):
    test=fgrp['Energy Layer [MeV]'].unique()[0]

In [30]:
print(test)
print(type(test))
fgrp['Energy Layer [MeV]'== np.float64(81.104)]

102.104
<class 'numpy.float64'>


KeyError: False

In [ ]:
pyesapi.clr.